In [1]:
import logging
import tempfile
from moviepy import *
import numpy as np
from pytubefix import YouTube
import cv2
from google.cloud import videointelligence
import io
import time
import json
import logging
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector
import os
import openai


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


c:\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
url = "https://www.youtube.com/watch?v=HBt86bgsgMU"
temp_dir = "D:\\Study\\Python\\Learning\\ClipKart\\New folder\\videos" 

In [3]:
yt = YouTube(url)

# Get the highest quality progressive stream
streams = yt.streams.filter(progressive=True, file_extension='mp4')
if not streams:
    raise ValueError("No suitable video streams found for this YouTube video.")
    
video = streams.order_by('resolution').desc().first()

if not video:
    raise ValueError("Could not find a suitable video format to download.")

In [4]:
#Captions
try: 
    caption = yt.captions['a.en']
except:
    caption = yt.captions['a.hi']

cap = caption.generate_srt_captions()
captions = cap.splitlines()
cap

"1\n00:00:00,240 --> 00:00:07,200\nthis is a video for wanted criminals\n\n2\n00:00:03,319 --> 00:00:10,440\nonly just turn yourself in i'm not in a\n\n3\n00:00:07,200 --> 00:00:13,719\npranking mood this April fools no horse\n\n4\n00:00:10,440 --> 00:00:17,279\nhockey i am using today for something\n\n5\n00:00:13,719 --> 00:00:19,920\nserious making a polite request for all\n\n6\n00:00:17,279 --> 00:00:23,840\nwanted criminals to go to the police\n\n7\n00:00:19,920 --> 00:00:27,199\nstation and turn yourself in pokemon go\n\n8\n00:00:23,840 --> 00:00:29,439\nto the station and let's put an end to\n\n9\n00:00:27,199 --> 00:00:31,519\nwhatever you've done if you're watching\n\n10\n00:00:29,439 --> 00:00:33,680\nthis footage and are not a wanted\n\n11\n00:00:31,519 --> 00:00:36,000\ncriminal please feel free to share this\n\n12\n00:00:33,680 --> 00:00:37,920\nvideo with a criminal you might know\n\n13\n00:00:36,000 --> 00:00:39,600\nperhaps it'll help them experience a\n\n14\n00:00:37,92

In [5]:
# Get video and audio streams
video = yt.streams.filter(res="1080p", mime_type="video/mp4").first()
audio = yt.streams.filter(only_audio=True).first()

output_dir = "D:\\Study\\Python\\Learning\\ClipKart\\New folder\\videos"

# Download video and audio to the output directory
video.download(output_path=output_dir, filename="video.mp4")
audio.download(output_path=output_dir, filename="audio.mp4")

# Define file paths
video_path = os.path.join(output_dir, "video.mp4")
audio_path = os.path.join(output_dir, "audio.mp4")
output_path = os.path.join(output_dir, "output.mp4")

# Merge video & audio using ffmpeg
os.system(f'ffmpeg -i "{video_path}" -i "{audio_path}" -c:v copy -c:a aac "{output_path}"')

# Remove temporary files
os.remove(video_path)
os.remove(audio_path)

In [ ]:
# import openai

# openai.api_key = "your-api-key"

# # Open the audio file
# audio_file = open("videos/audio.mp3", "rb")

# # Transcribe the audio and get an SRT-formatted response
# srt_transcription = openai.Audio.transcribe("whisper-1", audio_file, response_format="srt")

# # Save the SRT file
# with open("output.srt", "w", encoding="utf-8") as file:
#     file.write(srt_transcription)

# print("SRT file saved as output.srt")


In [17]:
# from moviepy.editor import VideoFileClip, AudioFileClip
# video = VideoFileClip("videos\output.mp4")



# square_video = video.crop(
#     width=1080, 
#     height=1080, 
#     x_center=video.w / 2,  # Crop from the center horizontally
#     y_center=video.h / 2    # Crop from the center vertically
# )

# # Step 2: Add black bars on top and bottom to make it 1080x1920
# padding = (1920 - 1080) // 2  # Calculate padding size (420px each)

# video_with_bars = square_video.margin(top=padding, bottom=padding, color=(0, 0, 0))

# # Save the final video
# video_with_bars.write_videofile("videos\output_square.mp4", codec="libx264", fps=video.fps)

2025-04-03 22:51:39,945 - WARNING - <>:2: SyntaxWarning: invalid escape sequence '\o'

2025-04-03 22:51:39,945 - WARNING - <>:17: SyntaxWarning: invalid escape sequence '\o'

2025-04-03 22:51:39,946 - WARNING - <>:2: SyntaxWarning: invalid escape sequence '\o'

2025-04-03 22:51:39,946 - WARNING - <>:17: SyntaxWarning: invalid escape sequence '\o'

2025-04-03 22:51:39,947 - WARNING - C:\Users\mrrpa\AppData\Local\Temp\ipykernel_8088\2629048891.py:2: SyntaxWarning: invalid escape sequence '\o'
  video = VideoFileClip("videos\output.mp4")

2025-04-03 22:51:39,948 - WARNING - C:\Users\mrrpa\AppData\Local\Temp\ipykernel_8088\2629048891.py:17: SyntaxWarning: invalid escape sequence '\o'
  video_with_bars.write_videofile("videos\output_square.mp4", codec="libx264", fps=video.fps)



Moviepy - Building video videos\output_square.mp4.
MoviePy - Writing audio in output_squareTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video videos\output_square.mp4



Moviepy - Done !
Moviepy - video ready videos\output_square.mp4


In [6]:
#Parse SRT INTO A LIST
def parse_srt(srt_text):
    blocks = srt_text.strip().split("\n\n")  # Split by caption blocks
    parsed_captions = []
    
    for block in blocks:
        lines = block.split("\n")
        if len(lines) >= 3:
            index = int(lines[0])  # Caption index
            time_range = lines[1]   # Time range
            text = " ".join(lines[2:])  # Caption text
            parsed_captions.append((index, time_range, text))
    
    return parsed_captions

captions_list = parse_srt(cap)

In [7]:

# Step 3: Convert time format from SRT to moviepy-compatible timestamps
def srt_to_seconds(time_str):
    h, m, s = time_str.split(":")
    s, ms = s.split(",")
    return int(h) * 3600 + int(m) * 60 + int(s) + int(ms) / 1000

formatted_captions = []
for index, time_range, text in captions_list:
    start, end = time_range.split(" --> ")
    start_time = srt_to_seconds(start)
    end_time = srt_to_seconds(end)
    formatted_captions.append((start_time, end_time, text))

In [61]:
# from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip


# # Step 4: Add captions to the video using moviepy
# video = VideoFileClip("videos\output.mp4")  # Load video file

# text_clips = []
# for start, end, text in formatted_captions:
#     txt_clip = TextClip(text, fontsize=24, color='white', bg_color='black')
#     txt_clip = txt_clip.set_position(("center", "bottom")).set_duration(end - start).set_start(start)
#     text_clips.append(txt_clip)

# # Overlay text on video
# final_video = CompositeVideoClip([video] + text_clips)

# square_video = final_video.crop(
#     width=1080, 
#     height=1080, 
#     x_center=final_video.w / 2,  # Crop from the center horizontally
#     y_center=final_video.h / 2    # Crop from the center vertically
# )

# # Step 2: Add black bars on top and bottom to make it 1080x1920
# padding = (1920 - 1080) // 2  # Calculate padding size (420px each)

# video_with_bars = square_video.margin(top=padding, bottom=padding, color=(0, 0, 0))

# # Save the final video
# video_with_bars.write_videofile("videos\output_square.mp4", codec="libx264", fps=video.fps)

2025-04-03 23:50:43,692 - WARNING - <>:5: SyntaxWarning: invalid escape sequence '\o'

2025-04-03 23:50:43,693 - WARNING - <>:29: SyntaxWarning: invalid escape sequence '\o'

2025-04-03 23:50:43,693 - WARNING - <>:5: SyntaxWarning: invalid escape sequence '\o'

2025-04-03 23:50:43,694 - WARNING - <>:29: SyntaxWarning: invalid escape sequence '\o'

2025-04-03 23:50:43,695 - WARNING - C:\Users\mrrpa\AppData\Local\Temp\ipykernel_8088\4069590584.py:5: SyntaxWarning: invalid escape sequence '\o'
  video = VideoFileClip("videos\output.mp4")  # Load video file

2025-04-03 23:50:43,696 - WARNING - C:\Users\mrrpa\AppData\Local\Temp\ipykernel_8088\4069590584.py:29: SyntaxWarning: invalid escape sequence '\o'
  video_with_bars.write_videofile("videos\output_square.mp4", codec="libx264", fps=video.fps)

2025-04-03 23:50:43,986 - WARNING - C:\Users\mrrpa\AppData\Local\Temp\ipykernel_8088\4069590584.py:5: SyntaxWarning: invalid escape sequence '\o'
  video = VideoFileClip("videos\output.mp4")  # Loa

OSError: MoviePy Error: creation of None failed because of the following error:

[WinError 2] The system cannot find the file specified.

.This error can be due to the fact that ImageMagick is not installed on your computer, or (for Windows users) that you didn't specify the path to the ImageMagick binary in file conf.py, or that the path you specified is incorrect

In [67]:
# from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip
# import moviepy.config as mpconf

# # Set the path to ImageMagick (change it based on your installation directory)
# mpconf.change_settings({"IMAGEMAGICK_BINARY": r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"})


# # Load video file
# video = VideoFileClip("videos/output.mp4")

# # Add captions
# text_clips = []
# for start, end, text in formatted_captions:
#     txt_clip = TextClip(text, fontsize=24, color='white', bg_color='black', font="Arial")
#     txt_clip = txt_clip.set_position(("center", "bottom")).set_duration(end - start).set_start(start)
#     text_clips.append(txt_clip)

# # Overlay text on video
# final_video = CompositeVideoClip([video] + text_clips)

# # Print video size to check
# print(f"Original video size: {final_video.w}x{final_video.h}")

# # Crop to square (centered)
# square_video = final_video.crop(
#     width=1080, 
#     height=1080, 
#     x_center=final_video.w / 2,  
#     y_center=final_video.h / 2    
# )

# # Add black bars for 1080x1920 format
# padding = (1080 - 1080) // 2  
# video_with_bars = square_video.margin(top=padding, bottom=padding, color=(0, 0, 0))

# # Save the final video
# video_with_bars.write_videofile("videos/output_square.mp4", codec="libx264", fps=video.fps)


2025-04-03 23:57:24,418 - WARNING - c:\Python\Lib\site-packages\moviepy\video\VideoClip.py:889: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imread(img)



Original video size: 1920x1080
Moviepy - Building video videos/output_square.mp4.
MoviePy - Writing audio in output_squareTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video videos/output_square.mp4



t:  96%|█████████▌| 2582/2684 [02:49<00:06, 15.99it/s, now=None]2025-04-04 00:00:15,894 - WARNING - c:\Python\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:123: UserWarning: Warning: in file videos/output.mp4, 6220800 bytes wanted but 0 bytes read,at frame 2581/2589, at time 43.02/43.14 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

2025-04-04 00:00:15,953 - WARNING - c:\Python\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:123: UserWarning: Warning: in file videos/output.mp4, 6220800 bytes wanted but 0 bytes read,at frame 2582/2589, at time 43.03/43.14 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  96%|█████████▋| 2584/2684 [02:49<00:06, 15.92it/s, now=None]2025-04-04 00:00:16,020 - WARNING - c:\Python\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:123: UserWarning: Warning: in file videos/output.mp4, 6220800 bytes wanted but 0 bytes read,at frame 2583/2589, at t

Moviepy - Done !
Moviepy - video ready videos/output_square.mp4


In [66]:
# from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip

# # Load video file
# video = VideoFileClip("videos/output.mp4")

# # Add captions using PIL (instead of ImageMagick)
# text_clips = []
# for start, end, text in formatted_captions:
#     txt_clip = TextClip(
#         text, fontsize=24, color='white', bg_color='black', font="Arial", method="caption"
#     )
#     txt_clip = txt_clip.set_position(("center", "bottom")).set_duration(end - start).set_start(start)
#     text_clips.append(txt_clip)

# # Overlay text on video
# final_video = CompositeVideoClip([video] + text_clips)

# # Crop to square (centered)
# square_video = final_video.crop(
#     width=1080, 
#     height=1080, 
#     x_center=final_video.w / 2,  
#     y_center=final_video.h / 2    
# )

# # Add black bars for 1080x1920 format
# padding = (1920 - 1080) // 2  
# video_with_bars = square_video.margin(top=padding, bottom=padding, color=(0, 0, 0))

# # Save the final video
# video_with_bars.write_videofile("videos/output_square.mp4", codec="libx264", fps=video.fps)


OSError: MoviePy Error: creation of None failed because of the following error:

[WinError 2] The system cannot find the file specified.

.This error can be due to the fact that ImageMagick is not installed on your computer, or (for Windows users) that you didn't specify the path to the ImageMagick binary in file conf.py, or that the path you specified is incorrect

In [78]:
# from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip

# import moviepy.config as mpconf

# # Set the path to ImageMagick (change it based on your installation directory)
# mpconf.change_settings({"IMAGEMAGICK_BINARY": r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"})

# # Load video file
# video = VideoFileClip("videos/output.mp4")

# text_clips = []
# for start, end, text in formatted_captions:
#     words = text.split()  # Split sentence into words
#     duration_per_word = (end - start) / len(words)  # Distribute time equally per word
    
#     for i, word in enumerate(words):
#         word_start = start + i * duration_per_word  # Set start time for each word
#         txt_clip = TextClip(
#             word, fontsize=100, color='white', font="Comic Sans MS", method="caption"
#         ).set_position(("center", "center")).set_duration(duration_per_word).set_start(word_start)
        
#         text_clips.append(txt_clip)

# # Overlay text on video
# final_video = CompositeVideoClip([video] + text_clips)

# # Crop to square (centered)
# square_video = final_video.crop(
#     width=1080, 
#     height=1080, 
#     x_center=final_video.w / 2,  
#     y_center=final_video.h / 2    
# )

# # Add black bars for 1080x1920 format
# padding = (1920 - 1080) // 2  
# video_with_bars = square_video.margin(top=padding, bottom=padding, color=(0, 0, 0))

# # Save the final video
# video_with_bars.write_videofile("videos/output_square.mp4", codec="libx264", fps=video.fps)


Moviepy - Building video videos/output_square.mp4.
MoviePy - Writing audio in output_squareTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video videos/output_square.mp4



t: 100%|█████████▉| 24036/24040 [32:53<00:00, 14.09it/s, now=None]2025-04-04 00:58:04,276 - WARNING - c:\Python\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:123: UserWarning: Warning: in file videos/output.mp4, 6220800 bytes wanted but 0 bytes read,at frame 24036/24040, at time 802.00/802.11 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

t: 100%|█████████▉| 24038/24040 [32:53<00:00, 13.69it/s, now=None]2025-04-04 00:58:04,353 - WARNING - c:\Python\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:123: UserWarning: Warning: in file videos/output.mp4, 6220800 bytes wanted but 0 bytes read,at frame 24037/24040, at time 802.03/802.11 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

2025-04-04 00:58:04,419 - WARNING - c:\Python\Lib\site-packages\moviepy\video\io\ffmpeg_reader.py:123: UserWarning: Warning: in file videos/output.mp4, 6220800 bytes wanted but 0 bytes read,at frame 240

Moviepy - Done !
Moviepy - video ready videos/output_square.mp4


In [ ]:
# from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip
# import moviepy.config as mpconf

# # Set ImageMagick binary path
# mpconf.change_settings({"IMAGEMAGICK_BINARY": r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"})

# # Load and trim video to first 30 seconds
# video = VideoFileClip("videos/output.mp4").subclip(0, 30)

# text_clips = []
# for start, end, text in formatted_captions:
#     if start > 30:  # Ignore captions beyond 30 seconds
#         break
    
#     words = text.split()  # Split sentence into words
#     duration_per_word = min((end - start) / len(words), 30 - start)  # Ensure within 30s

#     for i, word in enumerate(words):
#         word_start = start + i * duration_per_word  # Set start time for each word
#         if word_start >= 30:  # Stop adding words beyond 30 seconds
#             break

#         txt_clip = TextClip(
#             word, fontsize=100, color='white', font="Comic Sans MS", method="caption"
#         ).set_position(("center", "center")).set_duration(duration_per_word).set_start(word_start)
        
#         text_clips.append(txt_clip)

# # Overlay text on video
# final_video = CompositeVideoClip([video] + text_clips)

# # Crop to square (centered)
# square_video = final_video.crop(
#     width=1080, 
#     height=1080, 
#     x_center=final_video.w / 2,  
#     y_center=final_video.h / 2    
# )

# # Add black bars for 1080x1920 format
# padding = (1920 - 1080) // 2  
# video_with_bars = square_video.margin(top=padding, bottom=padding, color=(0, 0, 0))

# # Save the final video
# video_with_bars.write_videofile("videos/output_square.mp4", codec="libx264", fps=video.fps)


Moviepy - Building video videos/output_square.mp4.
MoviePy - Writing audio in output_squareTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video videos/output_square.mp4



Moviepy - Done !
Moviepy - video ready videos/output_square.mp4


In [ ]:
# from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip
# import moviepy.config as mpconf

# # Set ImageMagick binary path
# mpconf.change_settings({"IMAGEMAGICK_BINARY": r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"})

# # Load and trim video to first 30 seconds
# video = VideoFileClip("videos/output.mp4").subclip(0, 30)

# text_clips = []
# last_end_time = 0

# for start, end, text in formatted_captions:
#     if start >= 30:
#         break

#     # Prevent overlapping
#     actual_start = max(start, last_end_time)
#     actual_end = min(end, 30)

#     if actual_end <= actual_start:
#         continue  # Skip if duration is 0 or negative

#     duration = actual_end - actual_start

#     txt_clip = TextClip(
#         text,
#         fontsize=100,
#         color='white',
#         font="Comic Sans MS",
#         method="caption",
#         size=(video.w * 0.8, None),  # wrap text nicely
#         align='center'
#     ).set_position(("center", "bottom")).set_duration(duration).set_start(actual_start)

#     text_clips.append(txt_clip)
#     last_end_time = actual_start + duration

# # Overlay text on video
# final_video = CompositeVideoClip([video] + text_clips)

# # Crop to square (centered)
# square_video = final_video.crop(
#     width=1080, 
#     height=1080, 
#     x_center=final_video.w / 2,  
#     y_center=final_video.h / 2    
# )

# # Add black bars for 1080x1920 format
# padding = (1920 - 1080) // 2  
# video_with_bars = square_video.margin(top=padding, bottom=padding, color=(0, 0, 0))

# # Save the final video
# video_with_bars.write_videofile("videos/output_square.mp4", codec="libx264", fps=video.fps)


Moviepy - Building video videos/output_square.mp4.
MoviePy - Writing audio in output_squareTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video videos/output_square.mp4



Moviepy - Done !
Moviepy - video ready videos/output_square.mp4


In [10]:
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip
import moviepy.config as mpconf

# Set ImageMagick binary path
mpconf.change_settings({"IMAGEMAGICK_BINARY": r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"})

# Load and trim video to first 30 seconds
video = VideoFileClip("videos/output.mp4").subclip(0, 30)

text_clips = []
last_end_time = 0

for start, end, text in formatted_captions:
    if start >= 30:
        break

    # Prevent overlapping
    actual_start = max(start, last_end_time)
    actual_end = min(end, 30)

    if actual_end <= actual_start:
        continue  # Skip if duration is 0 or negative

    duration = actual_end - actual_start

    txt_clip = TextClip(
        text,
        fontsize=60,  # reduced for 1080p format
        color='white',
        font="Comic Sans MS",
        method="caption",
        size=(900, None),  # fits well within 1080px width
        align='center'
    ).set_position(("center", "bottom")).set_duration(duration).set_start(actual_start)

    text_clips.append(txt_clip)
    last_end_time = actual_start + duration

# Overlay text on video
final_video = CompositeVideoClip([video] + text_clips)

# Crop to square (centered)
square_video = final_video.crop(
    width=1080, 
    height=1080, 
    x_center=final_video.w / 2,  
    y_center=final_video.h / 2    
)

# Add black bars for 1080x1920 format
padding = (1920 - 1080) // 2  
video_with_bars = square_video.margin(top=padding, bottom=padding, color=(0, 0, 0))

# Save the final video
video_with_bars.write_videofile("videos/output_square.mp4", codec="libx264", fps=video.fps)


Moviepy - Building video videos/output_square.mp4.
MoviePy - Writing audio in output_squareTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video videos/output_square.mp4



Moviepy - Done !
Moviepy - video ready videos/output_square.mp4


In [ ]:
import logging
import os
import subprocess
from pytubefix import YouTube
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip
import moviepy.config as mpconf

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Paths
url = "https://www.youtube.com/watch?v=HBt86bgsgMU"
output_dir = os.path.join("D:", "Study", "Python", "Learning", "ClipKart", "New folder", "videos")
video_path = os.path.join(output_dir, "video.mp4")
audio_path = os.path.join(output_dir, "audio.mp4")
output_path = os.path.join(output_dir, "output.mp4")
final_output_path = os.path.join(output_dir, "output_square.mp4")

# Fetch video
yt = YouTube(url)
video_stream = yt.streams.filter(res="1080p", mime_type="video/mp4").first()
audio_stream = yt.streams.filter(only_audio=True).first()

if not video_stream or not audio_stream:
    raise ValueError("Video or audio stream not found.")

# Download video and audio
video_stream.download(output_path=output_dir, filename="video.mp4")
audio_stream.download(output_path=output_dir, filename="audio.mp4")

# Merge video and audio using ffmpeg (via subprocess)
subprocess.run([
    "ffmpeg",
    "-i", video_path,
    "-i", audio_path,
    "-c:v", "copy",
    "-c:a", "aac",
    output_path
], check=True)

# Clean up raw video/audio
os.remove(video_path)
os.remove(audio_path)

# Handle captions
caption = yt.captions.get_by_language_code('a.en') or yt.captions.get_by_language_code('a.hi')
if not caption:
    raise ValueError("No captions available in English or Hindi.")

srt_text = caption.generate_srt_captions()

# Parse SRT
def parse_srt(srt_text):
    blocks = srt_text.strip().split("\n\n")
    parsed = []
    for block in blocks:
        lines = block.split("\n")
        if len(lines) >= 3:
            index = int(lines[0])
            time_range = lines[1]
            text = " ".join(lines[2:])
            parsed.append((index, time_range, text))
    return parsed

captions_list = parse_srt(srt_text)

# Convert SRT time to seconds
def srt_to_seconds(time_str):
    h, m, s = time_str.split(":")
    s, ms = s.split(",")
    return int(h)*3600 + int(m)*60 + int(s) + int(ms)/1000

# Apply offset to captions
CAPTION_OFFSET = -3 # shift all subtitles back 1.5s for better alignment
formatted_captions = []
for index, time_range, text in captions_list:
    start, end = time_range.split(" --> ")
    start_time = max(srt_to_seconds(start) + CAPTION_OFFSET, 0)
    end_time = max(srt_to_seconds(end) + CAPTION_OFFSET, 0)
    formatted_captions.append((start_time, end_time, text))

# Set ImageMagick path (edit if needed)
mpconf.change_settings({"IMAGEMAGICK_BINARY": r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"})

# Load trimmed video
video = VideoFileClip(output_path).subclip(0, 30)

# Generate caption clips
text_clips = []
last_end_time = 0

for start, end, text in formatted_captions:
    if start >= 30:
        break
    actual_start = max(start, last_end_time)
    actual_end = min(end, 30)
    if actual_end <= actual_start:
        continue
    duration = actual_end - actual_start

    txt_clip = TextClip(
        text,
        fontsize=60,
        color='white',
        font="Arial",  # safer fallback
        method="caption",
        size=(900, None),
        align='center'
    ).set_position(("center", "bottom")).set_duration(duration).set_start(actual_start)

    text_clips.append(txt_clip)
    last_end_time = actual_start + duration

# Merge captions onto video
final_video = CompositeVideoClip([video] + text_clips)

# Crop to 1080x1080 (center)
square_video = final_video.crop(
    width=1080,
    height=1080,
    x_center=final_video.w / 2,
    y_center=final_video.h / 2
)

# Add black bars for 1080x1920 format
padding = (1920 - 1080) // 2
video_with_bars = square_video.margin(top=padding, bottom=padding, color=(0, 0, 0))

# Export
video_with_bars.write_videofile(final_output_path, codec="libx264", fps=video.fps)
